# sqlite3


Scripts to run database migration, table creation etc.

In [45]:
import sqlite3
import json

In [39]:
class dbopen(object):
    def __init__(self, path='data.db'):
        self.path = path
        
    def __enter__(self):
        self.conn = sqlite3.connect(self.path)
        self.cursor = self.conn.cursor()
        return self.cursor
    
    def __exit__(self, exc_class, exc, traceback):
        self.conn.commit()
        self.conn.close()

In [62]:
def create_table():
    with dbopen() as c:
        c.execute("""
        CREATE TABLE IF NOT EXISTS skill (
            id integer PRIMARY KEY AUTOINCREMENT,
            url text UNIQUE NOT NULL,
            text text NOT NULL,
            data json NOT NULL,
            created_at timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP,
            updated_at timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP
        )""")

        c.execute("""
        CREATE TRIGGER IF NOT EXISTS update_timestamp
        AFTER UPDATE ON skill 
        BEGIN
            update skill set updated_at = current_timestamp WHERE url = NEW.url;
        END
        """)

In [75]:
with dbopen('data.db') as c:
    result = c.execute('select id, url from skill').fetchall()
    print(result)

[(1, 'https://www.fullstackacademy.com/blog/nine-best-programming-languages-to-learn'), (2, 'https://www.ignite.digital/10-best-programming-languages-to-learn-in-2020/')]


# Preparing Training Data

In order to train the data for Spacy's NER, we first need to convert the format to the training data's format. We will load the data that is already stored in the database and change it to the following format:

```python
train_data = [
    ('React, Vue is good', {"entities": [(0, 5, 'SKILL'), (7, 10, 'SKILL')]})
]
```


References:
- https://spacy.io/usage/training
- https://www.machinelearningplus.com/nlp/training-custom-ner-model-in-spacy/

In [78]:
train_data = []
with dbopen() as c:
    result = c.execute('select id, data from skill').fetchall()
    for id, data in result:
        json_data = json.loads(data)
        for row in json_data:
            txt, annotations = row['text'], row['annotations']
            
            annotation_format = [(annotation['start'], 
                                  annotation['end'], 
                                  annotation['label']) for annotation in annotations]
            item_format = (txt, {"entities": annotation_format})
            train_data.append(item_format)

In [80]:
train_data[100]

('Some of our most well known and well-used apps are all built in Python; YouTube, Instagram, Pinterest and SurveyMonkey for example.',
 {'entities': [(64, 70, 'SKILL')]})

# Updating the Named Entity Recognizer

In [81]:
import spacy

# Load pre-existing spacy model.
nlp = spacy.load('en_core_web_sm')

# Getting the pipeline component.
ner = nlp.get_pipe('ner')

In [82]:
# Adding labels to the `ner`.
for _, annotations in train_data:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

In [84]:
# Apart from `ner`, the model has other pipeline components. 
# These components should not get affected in training.
# Disable the other pipeline components that you do not want to change.
# Training will be performed with the unaffected_pipes disabled

pipe_exceptions = ['ner', 'trf_wordpiecer', 'trf_tok2vec']
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
unaffected_pipes

['tagger', 'parser']

In [86]:
import random
from spacy.util import minibatch, compounding
from pathlib import Path

In [88]:
# Training the model.
with nlp.disable_pipes(*unaffected_pipes):
    

    for iteration in range(30): # 1
       
        random.shuffle(train_data) # 2
        losses = {}
        
        batches = minibatch(train_data, 
                            size=compounding(4.0, 32.0, 1.001)) # 3
            
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts,
                       annotations,
                       drop=0.5, # Make it harder to memorize data.
                       losses=losses)
            print('Losses:', losses)

Losses: {'ner': 265.239089012146}
Losses: {'ner': 365.98243618011475}
Losses: {'ner': 424.8524923324585}
Losses: {'ner': 513.2622573375702}
Losses: {'ner': 632.6993601322174}
Losses: {'ner': 707.4494364261627}
Losses: {'ner': 810.8481426239014}
Losses: {'ner': 901.7194237709045}
Losses: {'ner': 1024.5988655090332}
Losses: {'ner': 1195.0016222000122}


/Users/alextanhongpin/.local/share/virtualenvs/natural_language_processing_with_python_co-tJ8vfo8Q/lib/python3.8/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "It has lots of automation tools for testing and de..." with entities "[(220, 223, 'SKILL'), (287, 289, 'SKILL'), (333, 3...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)


Losses: {'ner': 1279.7422585487366}
Losses: {'ner': 1347.926778435707}
Losses: {'ner': 1509.7622438669205}
Losses: {'ner': 1592.0086470842361}
Losses: {'ner': 1681.8935877084732}


/Users/alextanhongpin/.local/share/virtualenvs/natural_language_processing_with_python_co-tJ8vfo8Q/lib/python3.8/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Best programming languages: C/C++C/C++ are the bac..." with entities "[(30, 33, 'SKILL'), (35, 38, 'SKILL')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)


Losses: {'ner': 1832.0886789560318}
Losses: {'ner': 1905.5773972272873}
Losses: {'ner': 1997.1453226804733}
Losses: {'ner': 2081.2259858846664}
Losses: {'ner': 2171.9830240011215}
Losses: {'ner': 2243.458684325218}
Losses: {'ner': 2293.963150382042}
Losses: {'ner': 2373.7914336919785}
Losses: {'ner': 2499.6672724485397}
Losses: {'ner': 2605.1758831739426}
Losses: {'ner': 2659.2568284273148}
Losses: {'ner': 2708.4451991319656}
Losses: {'ner': 2755.1527878046036}
Losses: {'ner': 2842.4960309267044}
Losses: {'ner': 2898.3096247911453}
Losses: {'ner': 2958.7948845624924}
Losses: {'ner': 3053.205299258232}
Losses: {'ner': 3144.20798265934}
Losses: {'ner': 3243.2644929885864}
Losses: {'ner': 3338.4397325515747}
Losses: {'ner': 3446.8466668128967}
Losses: {'ner': 3575.9023780822754}


/Users/alextanhongpin/.local/share/virtualenvs/natural_language_processing_with_python_co-tJ8vfo8Q/lib/python3.8/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Here is a quick summary of the different possibili..." with entities "[(140, 150, 'SKILL'), (152, 156, 'SKILL'), (158, 1...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)


Losses: {'ner': 3702.365786790848}
Losses: {'ner': 3771.3699753284454}
Losses: {'ner': 3850.5861723423004}
Losses: {'ner': 3922.5635039806366}
Losses: {'ner': 3953.6927428245544}
Losses: {'ner': 84.65512323379517}
Losses: {'ner': 172.10776495933533}
Losses: {'ner': 257.1546459197998}
Losses: {'ner': 319.8398151397705}
Losses: {'ner': 386.1344413757324}
Losses: {'ner': 545.5055055618286}
Losses: {'ner': 592.2638559341431}
Losses: {'ner': 682.8769421577454}
Losses: {'ner': 852.5059304237366}
Losses: {'ner': 912.3772773742676}
Losses: {'ner': 1002.7882475852966}
Losses: {'ner': 1074.3773349523544}
Losses: {'ner': 1140.6403383016586}
Losses: {'ner': 1206.7898887395859}
Losses: {'ner': 1306.5252867937088}
Losses: {'ner': 1385.737718462944}
Losses: {'ner': 1584.1527689695358}
Losses: {'ner': 1652.7162946462631}
Losses: {'ner': 1700.2855104207993}
Losses: {'ner': 1790.1166743040085}
Losses: {'ner': 1869.5237194299698}
Losses: {'ner': 1953.6327320337296}
Losses: {'ner': 2018.2706052064896}
Los

### Notes

1. To train the `ner` model, the model has to be looped over the example for sufficient number of iterations. If you train it for like just 5-6 iterations, it may not be effective. NOTE: find out what is the most effective number.
2. Before every iteration it's a good practice to shuffle the examples randomly through random.shuffle() function. 
This ensures that the model does not make generalization based on the order of the examples.
3. Training data is passed in batch. We use `minibatch()` function over the training data that will return data in batches. 

# Testing the model

In [98]:
doc = nlp('I want to learn Scala, go is also good.')
print('Entities', [(ent.text, ent.label_) 
                  for ent in doc.ents])

Entities [('Scala', 'SKILL'), ('go', 'SKILL')]


In [99]:
doc = nlp('can I learn js? Using a single threaded language like node.js is good for IO')
print('Entities', [(ent.text, ent.label_) 
                  for ent in doc.ents])

Entities []


# Saving the model

In [113]:
from datetime import datetime
today = datetime.now().strftime('%Y_%m_%d')
today

'2020_08_07'

In [114]:
import os
target_dir = f'content/{today}/'

if not os.path.exists(target_dir):
    os.makedirs(target_dir)
output_dir = Path(target_dir)
nlp.to_disk(output_dir)
print('Saved model to', output_dir)

Saved model to content/2020_08_07


# Loading the model and predict

In [115]:
print('Loading from', target_dir)

Loading from content/2020_08_07/


In [117]:
nlp_updated = spacy.load(output_dir)
doc = nlp_updated('Kotlin is an awesome programming language')
print('Entities', [(ent.text, ent.label_) 
                  for ent in doc.ents])

Entities [('Kotlin', 'SKILL')]
